In [3]:
import os
from dotenv import load_dotenv 
load_dotenv(override=True)

OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
print("OpenWeather API Key:", OPENWEATHER_API_KEY)

OpenWeather API Key: 1564bd59e86f981289a646fb2c421a63


In [4]:
import requests,json
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": OPENWEATHER_API_KEY,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)
# Example usage
get_weather("Beijing")

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\\u9634\\uff0c\\u591a\\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 29.49, "feels_like": 29.02, "temp_min": 29.49, "temp_max": 29.49, "pressure": 1007, "humidity": 39, "sea_level": 1007, "grnd_level": 1002}, "visibility": 10000, "wind": {"speed": 1.07, "deg": 163, "gust": 1.68}, "clouds": {"all": 100}, "dt": 1750908365, "sys": {"country": "CN", "sunrise": 1750884439, "sunset": 1750938415}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

## 将其封装为LangChain能够识别的外部函数

In [5]:
from langchain_core.tools import tool

@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": os.getenv("OPENWEATHER_API_KEY"),    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

- 测试

In [6]:
print(get_weather.name)
print(get_weather.description)
print(get_weather.args)

get_weather
查询即时天气函数
:param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
:return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
{'loc': {'title': 'Loc'}}


## 继续使用DeepSeek模型

In [7]:
from langchain.chat_models import init_chat_model

# 初始化模型
model = init_chat_model("deepseek-chat", model_provider="deepseek")

# 定义 天气查询 工具函数
tools = [get_weather]

# 将工具绑定到模型
llm_with_tools = model.bind_tools(tools)

response = llm_with_tools.invoke("你好， 请问北京的天气怎么样？")

print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_0_256fcbc4-7443-4625-98a8-3ff06872b432', 'function': {'arguments': '{"loc":"Beijing"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 203, 'total_tokens': 222, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 192}, 'prompt_cache_hit_tokens': 192, 'prompt_cache_miss_tokens': 11}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': 'a8688cee-5816-4d38-be0d-989f2f5d19a3', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run--46bc124a-ff25-41aa-b2bc-4f454a82c03a-0' tool_calls=[{'name': 'get_weather', 'args': {'loc': 'Beijing'}, 'id': 'call_0_256fcbc4-7443-4625-98a8-3ff06872b432', 'type': 'tool_call'}] usage_metadata={'input_tokens': 203, 'output_tokens': 19, 'total_tokens': 222, 'input_token_details': {'cache

In [8]:
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
# 使用JsonOutputKeyToolsParser解析工具调用的结果
# 只解析第一个工具调用的结果
parser = JsonOutputKeyToolsParser(key_name=get_weather.name, first_tool_only=True)
# 将模型和解析器组合成一个链
llm_chain = llm_with_tools | parser
# system = f"""
# 你可以访问一个名为 `df` 的 pandas 数据框，你可以使用df.head().to_markdown() 查看数据集的基本信息， \
# 请根据用户提出的问题，编写 Python 代码来回答。只返回代码，不返回其他内容。只允许使用 pandas 和内置库。
# """
prompt = ChatPromptTemplate([
    ("system", "你是天气助手，请根据用户的问题，给出相应的天气信息"),
    ("user", "{question}")
])
# Prompt 模板
output_prompt = PromptTemplate.from_template(
    """你将收到一段 JSON 格式的天气数据，请用简洁自然的方式将其转述给用户。
以下是天气 JSON 数据：

```json
{weather_json}
```
请将其转换为中文天气描述，例如：
“北京当前天气晴，气温为 23°C，湿度 58%，风速 2.1 米/秒。”
只返回一句话描述，不要其他说明或解释。"""
)
# 提问
get_weather_chain = prompt | llm_with_tools | parser | get_weather
# 将输出格式化为字符串
output_chain = output_prompt | model | StrOutputParser()
# 将两个链组合成一个完整的链
full_chain = get_weather_chain | output_chain
response = full_chain.invoke("请问北京今天的天气如何？")
print(response)

北京当前阴天多云，气温29.5°C，体感温度29°C，湿度39%，风速1.07米/秒。


## create_tool_calling_agent

In [9]:
from langchain.agents import create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate

#定义工具
tools = [get_weather]

# 构建提示模版
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是天气助手，请根据用户的问题，给出相应的天气信息"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# 初始化模型
model = init_chat_model("deepseek-chat", model_provider="deepseek")

# 直接使用`create_tool_calling_agent`创建代理
agent = create_tool_calling_agent(model, tools, prompt)
# Agent 是 LangChain 中用于处理工具调用的类
# 它可以根据用户输入和工具定义，自动选择合适的工具进行调用
# 将代理和工具组合成一个 AgentExecutor
# AgentExecutor 是 LangChain 中用于执行代理的类
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


response = agent_executor.invoke({"input": "请问今天北京的天气怎么样？"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'loc': 'Beijing'}`


{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 29.49, "feels_like": 29.02, "temp_min": 29.49, "temp_max": 29.49, "pressure": 1007, "humidity": 39, "sea_level": 1007, "grnd_level": 1002}, "visibility": 10000, "wind": {"speed": 1.07, "deg": 163, "gust": 1.68}, "clouds": {"all": 100}, "dt": 1750908423, "sys": {"country": "CN", "sunrise": 1750884439, "sunset": 1750938415}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}今天北京的天气是多云，阴天。当前温度为29.49°C，体感温度约为29.02°C。湿度为39%，风速为1.07米/秒，风向为163度。能见度为10公里。

> Finished chain.
{'input': '请问今天北京的天气怎么样？', 'output': '今天北京的天气是多云，阴天。当前温度为29.49°C，体感温度约为29.02°C。湿度为39%，风速为1.07米/秒，风向为163度。能见度为10公里。'}


In [10]:
# 一次性发起了同一个外部函数的两次调用请求，并依次获得了北京和杭州两个城市的天气。这就是一次标准的parallel_function_call 
# 并联示例
response = agent_executor.invoke({"input": "请问今天北京和杭州的天气怎么样，哪个城市更热？"})



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'loc': 'Beijing'}`


{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 29.49, "feels_like": 29.02, "temp_min": 29.49, "temp_max": 29.49, "pressure": 1007, "humidity": 39, "sea_level": 1007, "grnd_level": 1002}, "visibility": 10000, "wind": {"speed": 1.07, "deg": 163, "gust": 1.68}, "clouds": {"all": 100}, "dt": 1750908423, "sys": {"country": "CN", "sunrise": 1750884439, "sunset": 1750938415}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}
Invoking: `get_weather` with `{'loc': 'Hangzhou'}`


{"coord": {"lon": 120.1614, "lat": 30.2937}, "weather": [{"id": 803, "main": "Clouds", "description": "\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 33.68, "feels_like": 40.68, "temp_min": 33.68, "temp_max": 33.68, "pressure": 1008, "humidity": 67, "sea_lev

In [ ]:
# 此时我们再定义一个write_file函数，用于将“文本写入本地”：
@tool
def write_file(content):
    """
    将指定内容写入本地文件。
    :param content: 必要参数，字符串类型，用于表示需要写入文档的具体内容。
    :return：是否成功写入
    """
    
    return "已成功写入本地文件。"

from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate


tools = [get_weather, write_file]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是天气助手，请根据用户的问题，给出相应的天气信息，如果用户需要将查询结果写入文件，请使用write_file工具"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# 初始化模型
model = init_chat_model("deepseek-chat", model_provider="deepseek")

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#通过中间过程信息的打印，我们能够看到在一次交互过程中依次调用的get_weather查询到北京和杭州的天气，
# 然后又将结果写入到本地的文件中。这就是一个非常典型的串行工具调用的流程
agent_executor.invoke({"input": "查一下北京和杭州现在的温度，并将结果写入本地的文件中。"})